In [1]:
!nvidia-smi

Thu Nov 25 16:07:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.67       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:65:00.0 Off |                  N/A |
| 76%   63C    P2   151W / 370W |   2605MiB / 24265MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    Off  | 00000000:B3:00.0 Off |                  N/A |
| 76%   

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform as st
import os
import gc
import warnings
from sklearn.metrics import classification_report, average_precision_score
from Parse_TFrecords import *
from define_model import *
from load_data import *
from utilities import *
 
print(tf.__version__)

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

gpus = tf.config.list_physical_devices(device_type='GPU')
tf.config.set_visible_devices(devices=gpus[0], device_type='GPU')

2.5.0


In [3]:
pos = 27297
neg = 119133

total = pos+neg

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.61
Weight for class 1: 2.68


## Demographic attributes

In [4]:
checkpoint_filepath = 'checkpoints/AUC/checkpoint_demo_Dnet121'
monitor_ = 'val_auc'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    save_weights_only=True,
    monitor=monitor_,
    mode='max')

callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.EarlyStopping(mode='max', patience=3, monitor=monitor_),
            model_checkpoint_callback]

In [5]:
BATCH_SIZE = 32

record_file_train = 'copd_train_new.tfrecords'
train_dataset = (tf.data.TFRecordDataset(
    record_file_train, compression_type=None, buffer_size=32, num_parallel_reads=32)
.map(parse_TFrecord_train_demo)
.shuffle(total)
.batch(BATCH_SIZE))

record_file_val = 'copd_val_new.tfrecords'
val_dataset = (tf.data.TFRecordDataset(
    record_file_val, compression_type=None, buffer_size=32, num_parallel_reads=32)
.map(parse_TFrecord_train_demo)
.shuffle(36601)
.batch(BATCH_SIZE))

In [6]:
model = define_model_demo('Dnet121')

model.summary()

metric = tf.keras.metrics.AUC(name='auc')

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics=metric)

model.fit(train_dataset, epochs=20, shuffle=True, validation_data=val_dataset, callbacks=callback, class_weight=class_weight)

gc.collect()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_cxr (InputLayer)          [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0           input_cxr[0][0]                  
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/20
4576/4576 [==============================] - 284s 53ms/step - loss: 0.5015 - auc: 0.8331 - val_loss: 0.6253 - val_auc: 0.7513
Epoch 2/20
4576/4576 [==============================] - 275s 53ms/step - loss: 0.4916 - auc: 0.8403 - val_loss: 0.5867 - val_auc: 0.7502
Epoch 3/20
4576/4576 [==============================] - 276s 53ms/step - loss: 0.4901 - auc: 0.8415 - val_loss: 0.5957 - val_auc: 0.7500
Epoch 4/20
4576/4576 [==============================] - 274s 53ms/step - loss: 0.4895 - auc: 0.8418 - val_loss: 0.5869 - val_auc: 0.7495


539

In [5]:
checkpoint_filepath = 'checkpoints/AUC/checkpoint_demo_Dnet121'
model = define_model_demo()

model.load_weights(checkpoint_filepath)

test_image, test_demo, y_test = get_test_data_demo()

y_preds = model.predict([test_image, test_demo])

In [13]:
# 1034

test_CI(y_preds, y_test)

print(average_precision_score(y_test, y_preds, average=None))
    
thresh = get_thresh(y_test, y_preds, 'Youden')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

thresh = get_thresh(y_test, y_preds, 'G-mean')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

thresh = 0.5
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

gc.collect()

Sample auc mean: 0.75
Samole auc std: 0.00
Sample auc CI: 0.01
Confidence interval for AUC: [0.74 - 0.75]
0.4257054277679161
thresh: 0.43828025
              precision    recall  f1-score   support

           0       0.90      0.66      0.76     36808
           1       0.34      0.70      0.45      9026

    accuracy                           0.67     45834
   macro avg       0.62      0.68      0.61     45834
weighted avg       0.79      0.67      0.70     45834

thresh: 0.45863208
              precision    recall  f1-score   support

           0       0.90      0.68      0.78     36808
           1       0.34      0.67      0.46      9026

    accuracy                           0.68     45834
   macro avg       0.62      0.68      0.62     45834
weighted avg       0.79      0.68      0.71     45834

thresh: 0.5
              precision    recall  f1-score   support

           0       0.89      0.72      0.80     36808
           1       0.36      0.63      0.46      9026

    acc

1820

In [6]:
for i in range(2):

    test_image, test_demo, y_test = get_test_data_demo(category='Gender', types=i)

    y_preds = model.predict([test_image, test_demo])

    test_CI(y_preds, y_test)

    print(average_precision_score(y_test, y_preds, average=None))

    thresh = get_thresh(y_test, y_preds, 'Youden')
    print('thresh:', thresh)

    print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

    thresh = get_thresh(y_test, y_preds, 'G-mean')
    print('thresh:', thresh)

    print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

    thresh = 0.5
    print('thresh:', thresh)

    print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))


    del test_image, test_demo, y_test, y_preds
    gc.collect()

Sample auc mean: 0.76
Samole auc std: 0.00
Sample auc CI: 0.01
Confidence interval for AUC: [0.76 - 0.77]
0.43592311781905585
thresh: 0.5421198
              precision    recall  f1-score   support

           0       0.90      0.75      0.82     17137
           1       0.38      0.65      0.48      4032

    accuracy                           0.73     21169
   macro avg       0.64      0.70      0.65     21169
weighted avg       0.80      0.73      0.75     21169

thresh: 0.5165307
              precision    recall  f1-score   support

           0       0.90      0.72      0.80     17137
           1       0.36      0.67      0.47      4032

    accuracy                           0.71     21169
   macro avg       0.63      0.70      0.64     21169
weighted avg       0.80      0.71      0.74     21169

thresh: 0.5
              precision    recall  f1-score   support

           0       0.91      0.71      0.79     17137
           1       0.36      0.69      0.47      4032

    accu

In [9]:
for i in range(5):

    test_image, test_demo, y_test = get_test_data_demo(category='Race', types=i)

    y_preds = model.predict([test_image, test_demo])

    test_CI(y_preds, y_test)

    print(average_precision_score(y_test, y_preds, average=None))

    thresh = get_thresh(y_test, y_preds, 'Youden')
    print('thresh:', thresh)

    print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

    thresh = get_thresh(y_test, y_preds, 'G-mean')
    print('thresh:', thresh)

    print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

    thresh = 0.5
    print('thresh:', thresh)

    print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

    del test_image, test_demo, y_test, y_preds
    gc.collect()

Sample auc mean: 0.74
Samole auc std: 0.00
Sample auc CI: 0.01
Confidence interval for AUC: [0.73 - 0.75]
0.4265995335736654
thresh: 0.4625675
              precision    recall  f1-score   support

           0       0.90      0.65      0.75     23886
           1       0.33      0.70      0.45      6011

    accuracy                           0.66     29897
   macro avg       0.61      0.67      0.60     29897
weighted avg       0.78      0.66      0.69     29897

thresh: 0.482695
              precision    recall  f1-score   support

           0       0.89      0.67      0.76     23886
           1       0.34      0.68      0.45      6011

    accuracy                           0.67     29897
   macro avg       0.62      0.67      0.61     29897
weighted avg       0.78      0.67      0.70     29897

thresh: 0.5
              precision    recall  f1-score   support

           0       0.89      0.68      0.77     23886
           1       0.34      0.66      0.45      6011

    accura

In [10]:
for i in range(4):

    test_image, test_demo, y_test = get_test_data_demo(category='Age', types=i)

    y_preds = model.predict([test_image, test_demo])

    test_CI(y_preds, y_test)

    print(average_precision_score(y_test, y_preds, average=None))

    thresh = get_thresh(y_test, y_preds, 'Youden')
    print('thresh:', thresh)

    print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

    thresh = get_thresh(y_test, y_preds, 'G-mean')
    print('thresh:', thresh)

    print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

    thresh = 0.5
    print('thresh:', thresh)

    print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

    del test_image, test_demo, y_test, y_preds
    gc.collect()

Sample auc mean: 0.72
Samole auc std: 0.02
Sample auc CI: 0.04
Confidence interval for AUC: [0.68 - 0.76]
0.1454375604024976
thresh: 0.046171676
              precision    recall  f1-score   support

           0       0.98      0.55      0.71      4667
           1       0.07      0.79      0.14       214

    accuracy                           0.56      4881
   macro avg       0.53      0.67      0.42      4881
weighted avg       0.94      0.56      0.68      4881

thresh: 0.08568344
              precision    recall  f1-score   support

           0       0.98      0.70      0.81      4667
           1       0.09      0.63      0.15       214

    accuracy                           0.69      4881
   macro avg       0.53      0.66      0.48      4881
weighted avg       0.94      0.69      0.78      4881

thresh: 0.5
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      4667
           1       0.19      0.12      0.15       214

    ac